In [127]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as  mpl

In [128]:
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

In [129]:
def readData(y,s,c):
    df = pd.read_csv("lvr_land/"+ y +"-s" + s + "/" + c + "_lvr_land_A.csv",encoding='utf-8',low_memory=False)
    return df

In [130]:
def combineDF(df1,df2):
    df = pd.concat([df1,df2],axis = 0)
    return df

In [131]:
def selectFarmland(df):
    
    df = df[["鄉鎮市區","土地移轉總面積(平方公尺)","交易標的","都市土地使用分區","非都市土地使用分區","非都市土地使用編定","總價(元)","單價(元/平方公尺)"]]
    df_farmland = df[((df.交易標的 == "土地")&(((df.都市土地使用分區 =="農")&(df.非都市土地使用編定 !="丁種建築用地"))| ((df.非都市土地使用分區 == "一般農業區")& (df.非都市土地使用編定 !="丁種建築用地"))|((df.非都市土地使用分區 == '特定農業區')& (df.非都市土地使用編定 !="丁種建築用地")))) ]
    
    return df_farmland

In [132]:
def fillNanprice(df_farmland):
    #modify zero or nan price
    df_ok = df_farmland[(df_farmland["單價(元/平方公尺)"].notnull())&(df_farmland["單價(元/平方公尺)"] != "0")]
    df = df_farmland[(df_farmland["單價(元/平方公尺)"].isnull())|(df_farmland["單價(元/平方公尺)"] == "0")]
    df = df[(df['土地移轉總面積(平方公尺)'] != "0")&(df['總價(元)'] != "0")]
    df["單價(元/平方公尺)"] = calculateprice(df['總價(元)'],df['土地移轉總面積(平方公尺)'])
    df_farmland = pd.concat([df_ok,df],axis = 0)
    return df_farmland

In [133]:
def grouptown(df,list_all_town):
    grouped=pd.to_numeric(df_farmland['單價(元/平方公尺)']).groupby(df_farmland['鄉鎮市區'])
    town_dict = {}
   
    for key,group in grouped:
        m = grouped.count()
        town_dict[key]= m[key] 
        if key not in list_all_town:
            list_all_town.append(key)
    return town_dict


In [134]:
def transcationNum(df):
    return df['鄉鎮市區'].count()

In [135]:
def calculateprice(total_price,area):
    return pd.to_numeric(total_price)/pd.to_numeric(area)

In [136]:
def totalcalculate(key): 
    value = sum(key.values())   
    return value

In [137]:
def averagenum(num):
    nsum = 0
    for i in num:
        nsum = nsum + i
    return nsum / len(num)

In [138]:
list_cityfilename = ['A','B','C','D','E','F','G','H','I','J','K','M','N','O','P','Q','T','U','V','W','X','Z']
list_city = ['台北市','台中市','基隆市','台南市','高雄市','新北市','宜蘭縣','桃園縣','嘉義市','新竹縣','苗栗縣','南投縣','彰化縣','新竹市','雲林縣','嘉義縣','屏東縣','花蓮縣','台東縣','金門縣','澎湖縣','連江縣']
#list_cityfilename = ['H','D','N','B','T']
#list_city = ['桃園縣','台南市','彰化縣','台中市','屏東縣']
i=0
taiwan_transacation = {}#各縣市各年交易量
for c in list_cityfilename: #every city
    
    s = 4 # from 101-s4 to 107-s2
    town_tran_dist = {}
    鄉鎮逐年交易量 = {}
    list_all_town = []
    for y in range(101,108): #every year
        df = pd.DataFrame()
        while s <= 4:
            # read data
            df_s =  readData(str(y),str(s),c)
            df = combineDF(df,df_s)

            if y == 107 and s == 2:
                break
            s = s+1
        s=1
        # select farmland
        df_farmland = selectFarmland(df)
        # fill nan price
        df_farmland = fillNanprice(df_farmland)
        #鄉鎮各年交易量
        鄉鎮逐年交易量[str(y)] = grouptown(df_farmland,list_all_town)
        #縣市各年交易量
        town_tran_dist[y] = transcationNum(df_farmland)
        
    taiwan_transacation[list_city[i]]=town_tran_dist
   
    #各鄉鎮101-107總交易量
  
    鄉鎮={}
    鄉鎮歷年交易量 = {}
    各鄉鎮加總 = {}
    for t in list_all_town:
        鄉鎮 = {}
        for y in range(101,108):
            if t in 鄉鎮逐年交易量[str(y)].keys():
                鄉鎮[y] = 鄉鎮逐年交易量[str(y)][t]
        鄉鎮歷年交易量[t] = 鄉鎮
        各鄉鎮加總[t]=totalcalculate(鄉鎮歷年交易量[t])
        
    #鄉鎮總交易量排名
    sorted_transcation = sorted(各鄉鎮加總.items(),key=lambda s:s[1], reverse=True)
    
    for s in range(len(list_all_town)):
        print("|"+str(s+1)+"|",end='')
        print(sorted_transcation[s][0]+"/"+str(sorted_transcation[s][1])+"|") 
    i = i+1
#     total_transcation = {}
#     for c in list_city:
#         total_transcation[c] = totalcalculate(taiwan_transacation[c])
#     print(total_transcation)

#     table_dict = {}
#     sorted_transcation = sorted(total_transcation.items(),key=lambda s:s[1], reverse=True)
#     print("")
#     print(sorted_transcation)
#     print("")
#     for i in range(0,22):
#         print("|"+str(i+1)+"|",end='')
#         print(sorted_transcation[i][0]+"|"+str(sorted_transcation[i][1])+"|") 



|1|北投區/414|
|2|士林區/58|
|3|文山區/30|
|4|中山區/7|
|5|內湖區/5|
|6|南港區/4|
|1|后里區/1234|
|2|霧峰區/1229|
|3|大甲區/1218|
|4|神岡區/1196|
|5|烏日區/1053|
|6|豐原區/1005|
|7|大雅區/933|
|8|外埔區/881|
|9|清水區/829|
|10|潭子區/775|
|11|新社區/742|
|12|西屯區/685|
|13|大里區/662|
|14|大肚區/571|
|15|大安區/556|
|16|東勢區/548|
|17|龍井區/522|
|18|南屯區/509|
|19|沙鹿區/369|
|20|北屯區/333|
|21|石岡區/330|
|22|太平區/327|
|23|梧棲區/192|
|24|和平區/35|
|25|北區/2|
|26|南區/2|
|27|東區/2|
|1|七堵區/16|
|2|暖暖區/10|
|3|安樂區/1|
|1|安南區/1520|
|2|關廟區/1313|
|3|歸仁區/1296|
|4|善化區/1112|
|5|新化區/1032|
|6|白河區/1029|
|7|仁德區/997|
|8|七股區/987|
|9|佳里區/986|
|10|安定區/981|
|11|麻豆區/963|
|12|後壁區/934|
|13|下營區/923|
|14|西港區/846|
|15|將軍區/822|
|16|鹽水區/749|
|17|新市區/737|
|18|柳營區/737|
|19|官田區/724|
|20|東山區/629|
|21|永康區/618|
|22|學甲區/518|
|23|六甲區/515|
|24|新營區/497|
|25|南區/392|
|26|大內區/375|
|27|玉井區/309|
|28|北門區/296|
|29|左鎮區/276|
|30|山上區/244|
|31|楠西區/225|
|32|南化區/25|
|33|龍崎區/2|
|34|安平區/1|
|1|大寮區/1969|
|2|美濃區/1762|
|3|路竹區/937|
|4|大樹區/884|
|5|岡山區/831|
|6|阿蓮區/827|
|7|旗山區/743|
|8|仁武區/567|
|9|燕巢區/515|
|10|大社區/513|
|11|鳥松區/50

In [139]:
y_taiwan_transcation = {}
for y in range(101,108):
    年度交易量 = []
    for c in list_city:      
        年度交易量.append(taiwan_transacation[c][y])
    y_taiwan_transcation[y] = sum(年度交易量)

#平均交易量
print("平均交易量")
print(averagenum(y_taiwan_transcation.values()))

for i in range(101,108):
    print("|"+str(i)+"|",end='')
    print(str(y_taiwan_transcation[i])+"|") 


平均交易量
28786.14285714286
|101|8312|
|102|48748|
|103|42934|
|104|33996|
|105|25669|
|106|28268|
|107|13576|


In [140]:
#縣市各年交易量排名
print("縣市各年交易量排名")
table_dict = {}
for y in range(101,108):
    sorted_transcation = sorted(taiwan_transacation.items(),key=lambda s:s[1][y], reverse=True)
    table_dict[y] = sorted_transcation
for i in range(0,22):
    print("|"+str(i+1)+"|",end='')
    for y in range(101,108):
        print(table_dict[y][i][0]+"/"+str(table_dict[y][i][1][y])+"|",end="") 
    print("")

縣市各年交易量排名
|1|桃園縣/1760|桃園縣/10769|桃園縣/6479|台南市/4119|台南市/3114|台南市/4047|台南市/1774|
|2|台中市/815|台南市/4924|台南市/4896|彰化縣/3540|桃園縣/2942|桃園縣/3292|桃園縣/1487|
|3|台南市/736|台中市/4185|彰化縣/3914|桃園縣/3536|彰化縣/2648|彰化縣/3112|雲林縣/1417|
|4|屏東縣/721|彰化縣/4042|台中市/3780|屏東縣/2937|雲林縣/2482|屏東縣/2825|彰化縣/1391|
|5|彰化縣/658|高雄市/3181|屏東縣/3108|台中市/2824|屏東縣/2438|雲林縣/2757|屏東縣/1241|
|6|高雄市/521|屏東縣/3131|宜蘭縣/3074|雲林縣/2579|台中市/1832|台中市/2261|南投縣/1173|
|7|雲林縣/473|雲林縣/2590|高雄市/2716|高雄市/2331|高雄市/1741|高雄市/2170|台中市/1043|
|8|嘉義縣/380|新竹縣/2075|雲林縣/2704|宜蘭縣/2031|嘉義縣/1621|嘉義縣/1667|高雄市/926|
|9|新竹縣/371|南投縣/2053|花蓮縣/2178|嘉義縣/1791|宜蘭縣/1129|南投縣/1017|嘉義縣/775|
|10|南投縣/338|宜蘭縣/1967|嘉義縣/1795|花蓮縣/1598|南投縣/1105|宜蘭縣/927|宜蘭縣/414|
|11|宜蘭縣/317|嘉義縣/1847|南投縣/1742|南投縣/1551|花蓮縣/845|苗栗縣/838|新竹縣/338|
|12|花蓮縣/311|苗栗縣/1813|苗栗縣/1518|苗栗縣/1154|苗栗縣/797|花蓮縣/715|苗栗縣/325|
|13|苗栗縣/265|花蓮縣/1685|新竹縣/1469|新竹縣/1093|新竹縣/787|新竹縣/655|花蓮縣/312|
|14|新北市/192|新北市/1200|新北市/1065|新北市/968|澎湖縣/660|新北市/547|新北市/263|
|15|金門縣/110|澎湖縣/938|澎湖縣/706|澎湖縣/670|新北市/619|澎湖縣/431|澎湖縣/204|
|16|台東縣/104|金門縣

In [141]:
#各縣市101-107總交易量
print("各縣市101-107總交易量")
total_transcation = {}
for c in list_city:
    total_transcation[c] = totalcalculate(taiwan_transacation[c])

table_dict = {}
sorted_transcation = sorted(total_transcation.items(),key=lambda s:s[1], reverse=True)
print("")
print(sorted_transcation)
print("")
for i in range(0,22):
    print("|"+str(i+1)+"|",end='')
    print(sorted_transcation[i][0]+"|"+str(sorted_transcation[i][1])+"|") 

各縣市101-107總交易量
{'台北市': 518, '台中市': 16740, '基隆市': 27, '台南市': 23610, '高雄市': 13586, '新北市': 4854, '宜蘭縣': 9859, '桃園縣': 30265, '嘉義市': 879, '新竹縣': 6788, '苗栗縣': 6710, '南投縣': 8979, '彰化縣': 19305, '新竹市': 1253, '雲林縣': 15002, '嘉義縣': 9876, '屏東縣': 16401, '花蓮縣': 7644, '台東縣': 3081, '金門縣': 2410, '澎湖縣': 3709, '連江縣': 7}

[('桃園縣', 30265), ('台南市', 23610), ('彰化縣', 19305), ('台中市', 16740), ('屏東縣', 16401), ('雲林縣', 15002), ('高雄市', 13586), ('嘉義縣', 9876), ('宜蘭縣', 9859), ('南投縣', 8979), ('花蓮縣', 7644), ('新竹縣', 6788), ('苗栗縣', 6710), ('新北市', 4854), ('澎湖縣', 3709), ('台東縣', 3081), ('金門縣', 2410), ('新竹市', 1253), ('嘉義市', 879), ('台北市', 518), ('基隆市', 27), ('連江縣', 7)]

|1|桃園縣|30265|
|2|台南市|23610|
|3|彰化縣|19305|
|4|台中市|16740|
|5|屏東縣|16401|
|6|雲林縣|15002|
|7|高雄市|13586|
|8|嘉義縣|9876|
|9|宜蘭縣|9859|
|10|南投縣|8979|
|11|花蓮縣|7644|
|12|新竹縣|6788|
|13|苗栗縣|6710|
|14|新北市|4854|
|15|澎湖縣|3709|
|16|台東縣|3081|
|17|金門縣|2410|
|18|新竹市|1253|
|19|嘉義市|879|
|20|台北市|518|
|21|基隆市|27|
|22|連江縣|7|
